In [13]:
import logging as log
import numpy as np
from ortools.linear_solver import pywraplp

### Set up logging


In [14]:
for handler in log.root.handlers[:]:
    log.root.removeHandler(handler)
log.basicConfig(format="%(levelname)s - %(message)s")

logger = log.getLogger()
logger.setLevel(log.DEBUG)
log.info("Logger initialized...")

INFO - Logger initialized...


### Get input


In [15]:
def getinput(file):
    with open(file) as data:
        NumNode = int(next(data))
        PathCost = np.loadtxt(data)
    return NumNode, PathCost

In [16]:
NumNode, PathCost = getinput("largeTSP.txt")

### Set up solver


In [17]:
solver_id = "SCIP"
solver: pywraplp.Solver = pywraplp.Solver.CreateSolver(solver_id)
inf = solver.infinity()

### Generating decision variables


In [18]:
log.info("Creating " + str(NumNode * NumNode) + " boolean x_ij variables... ")
X = {}
for i in range(NumNode):
    for j in range(NumNode):
        X[(i, j)] = solver.BoolVar(f"x{(i, j)}")

INFO - Creating 900 boolean x_ij variables... 


### Creating constraints


In [19]:
# Constraint 1: leave every point exactly once

log.info("Creating " + str(NumNode) + " Constraint 1... ")
for i in range(NumNode):
    solver.Add(sum(X[(i, j)] for j in range(NumNode) if i != j) == 1)

INFO - Creating 30 Constraint 1... 


In [20]:
# Constraint 2: reach every point from exactly one other point

log.info("Creating " + str(NumNode) + " Constraint 2... ")
for j in range(NumNode):
    solver.Add(sum(X[(i, j)] for i in range(NumNode) if i != j) == 1)

INFO - Creating 30 Constraint 2... 


In [21]:
U = {}
U[0] = solver.IntVar(1, 1, "u0")
for i in range(1, NumNode):
    U[i] = solver.IntVar(2, NumNode, f"u{i}")

# constraint 3: subtour elimination constraints (Miller-Tucker-Zemlin)

log.info("Creating " + str((NumNode - 1) ** 2) + " Constraint 3... ")
for i in range(1, NumNode):
    for j in range(1, NumNode):
        solver.Add(U[i] - U[j] + 1 <= (NumNode - 1) * (1 - X[(i, j)]))

INFO - Creating 841 Constraint 3... 


### Define objective function and solve


In [22]:
# Create objective function
solver.Minimize(
    sum(X[(i, j)] * PathCost[i][j] 
    for i in range(NumNode) 
    for j in range(NumNode))
)
log.info("Solving MIP model... ")

INFO - Solving MIP model... 


### Print solution


In [23]:
def print_solution(X):
    start = 0
    print(start + 1, end=" ")
    for step in range(NumNode - 1):
        for end in range(NumNode):
            if X[(start, end)].solution_value() == 1:
                print("->", end + 1, end = " ")
                start = end
                break
    print("->", 1)

In [24]:
status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    print("Optimal objective value ==", solver.Objective().Value())
    print_solution(X)
else:
    print("UNBOUNDED")

Optimal objective value == 43399.0
1 -> 16 -> 2 -> 11 -> 17 -> 12 -> 13 -> 10 -> 28 -> 4 -> 26 -> 20 -> 8 -> 6 -> 19 -> 5 -> 9 -> 22 -> 7 -> 24 -> 21 -> 18 -> 27 -> 29 -> 14 -> 30 -> 23 -> 25 -> 15 -> 3 -> 1
